STAT 453: Deep Learning (Spring 2021)  
Instructor: Sebastian Raschka (sraschka@wisc.edu)  

Course website: http://pages.stat.wisc.edu/~sraschka/teaching/stat453-ss2021/  
GitHub repository: https://github.com/rasbt/stat453-deep-learning-ss21

---

In [1]:
!pip install watermark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Author: Sebastian Raschka

Python implementation: CPython
Python version       : 3.7.15
IPython version      : 7.9.0

torch: 1.12.1+cu113



# MLP with Dropout 

## Imports

In [3]:
!git clone https://github.com/andrei-radulescu-banu/stat453-deep-learning-ss21.git

fatal: destination path 'stat453-deep-learning-ss21' already exists and is not an empty directory.


In [4]:
!ls /content/stat453-deep-learning-ss21/L12/code

adabelief.ipynb       batchsize-64.ipynb    helper_train.py
adam.ipynb	      helper_dataset.py     __pycache__
adamW.ipynb	      helper_evaluation.py  scheduler.ipynb
batchsize-1024.ipynb  helper_plotting.py    sgd-scheduler-momentum.ipynb


In [5]:
import sys, os
sys.path.append("/content/stat453-deep-learning-ss21/L12/code")

In [6]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# From local helper files
from helper_evaluation import set_all_seeds, set_deterministic
from helper_train import train_model
from helper_plotting import plot_training_loss, plot_accuracy, show_examples
from helper_dataset import get_dataloaders_mnist

## Settings and Dataset

In [8]:
##########################
### SETTINGS
##########################

RANDOM_SEED = 123
BATCH_SIZE = 256
NUM_HIDDEN_1 = 75
NUM_HIDDEN_2 = 45
NUM_EPOCHS = 100
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [9]:
set_all_seeds(RANDOM_SEED)
#set_deterministic()

In [10]:
##########################
### MNIST DATASET
##########################

train_loader, valid_loader, test_loader = get_dataloaders_mnist(
    batch_size=BATCH_SIZE,
    validation_fraction=0.1)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])
Class labels of 10 examples: tensor([4, 5, 8, 9, 9, 4, 9, 9, 3, 9])


## Model

In [11]:
class MultilayerPerceptron(torch.nn.Module):

    def __init__(self, num_features, num_classes, drop_proba, 
                 num_hidden_1, num_hidden_2):
        super().__init__()
        
        self.my_network = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Flatten(),
            torch.nn.Linear(num_features, num_hidden_1, bias=False),
            torch.nn.BatchNorm1d(num_hidden_1),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            # 2nd hidden layer
            torch.nn.Linear(num_hidden_1, num_hidden_2, bias=False),
            torch.nn.BatchNorm1d(num_hidden_2),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            # output layer
            torch.nn.Linear(num_hidden_2, num_classes)
        )
           
    def forward(self, x):
        logits = self.my_network(x)
        return logits

In [ ]:
torch.manual_seed(RANDOM_SEED)
model = MultilayerPerceptron(num_features=28*28,
                             num_hidden_1=NUM_HIDDEN_1,
                             num_hidden_2=NUM_HIDDEN_2,
                             drop_proba=0.5,
                             num_classes=10)
model = model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

minibatch_loss_list, train_acc_list, valid_acc_list = train_model(
    model=model,
    num_epochs=NUM_EPOCHS,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=DEVICE,
    logging_interval=100)

plot_training_loss(minibatch_loss_list=minibatch_loss_list,
                   num_epochs=NUM_EPOCHS,
                   iter_per_epoch=len(train_loader),
                   results_dir=None,
                   averaging_iterations=20)
plt.show()

plot_accuracy(train_acc_list=train_acc_list,
              valid_acc_list=valid_acc_list,
              results_dir=None)
plt.ylim([80, 100])
plt.show()

Epoch: 001/100 | Batch 0000/0210 | Loss: 2.3859
Epoch: 001/100 | Batch 0100/0210 | Loss: 0.4805
Epoch: 001/100 | Batch 0200/0210 | Loss: 0.4440
Epoch: 001/100 | Train: 93.69% | Validation: 95.07%
Time elapsed: 0.22 min
Epoch: 002/100 | Batch 0000/0210 | Loss: 0.4458
Epoch: 002/100 | Batch 0100/0210 | Loss: 0.3063
Epoch: 002/100 | Batch 0200/0210 | Loss: 0.3967
Epoch: 002/100 | Train: 95.52% | Validation: 96.35%
Time elapsed: 0.39 min
Epoch: 003/100 | Batch 0000/0210 | Loss: 0.3934
Epoch: 003/100 | Batch 0100/0210 | Loss: 0.2613
Epoch: 003/100 | Batch 0200/0210 | Loss: 0.3276
Epoch: 003/100 | Train: 95.88% | Validation: 96.22%
Time elapsed: 0.55 min
Epoch: 004/100 | Batch 0000/0210 | Loss: 0.2734
Epoch: 004/100 | Batch 0100/0210 | Loss: 0.3736
Epoch: 004/100 | Batch 0200/0210 | Loss: 0.2961
Epoch: 004/100 | Train: 96.43% | Validation: 96.73%
Time elapsed: 0.72 min
Epoch: 005/100 | Batch 0000/0210 | Loss: 0.2352
Epoch: 005/100 | Batch 0100/0210 | Loss: 0.2915
Epoch: 005/100 | Batch 0200/